## データについて
各カラムの意味は以下の通りです.
<table>
  <thead>
  <tr>
    <td>.csvファイル</td>
    <td>学習モデル</td>
    <td>CVスコア</td>
    <td>暫定スコア</td>
    <td>備考</td>     
  </tr>
  </thead>
  <tr>
    <td>submission06102200.csv</td>
    <td>LR only</td> 
    <td>0.87363</td>
    <td>0.86669</td>
    <td>officer_idは100000のみ</td>
  </tr>
  <tr>
    <td>submission06111044.csv</td>
    <td>{LR & RF(n=100)}/2</td> 
    <td>0.87809</td>
    <td>0.?????</td>
    <td>同上</td>
  </tr>
  <tr>
    <td>submission06111100.csv</td>
    <td>{LR & RF(n=100)}/2</td> 
    <td>0.88333</td>
    <td>0.?????</td>
    <td>stop_hourを修正</td>
  </tr> 
  <tr>
    <td>なし</td>
    <td>XGB(深さ5)</td> 
    <td>0.88728</td>
    <td>0.?????</td>
    <td></td>
  </tr>  
  <tr>
    <td>なし</td>
    <td>XGB(深さ4,n_est=200)</td> 
    <td>0.88848</td>
    <td>0.?????</td>
    <td></td>
  </tr>   
  <tr>
    <td>なし</td>
    <td>XGB(深さ5)とLRを10:1</td> 
    <td>0.88766</td>
    <td>0.?????</td>
    <td></td>
  </tr>  
  <tr>
    <td>なし</td>
    <td>XGB(深さ4)</td> 
    <td>0.86897</td>
    <td>0.?????</td>
    <td>officer_idを追加した</td>
  </tr>  
  <tr>
    <td>submission_06111430.csv</td>
    <td>XGB(確か4と400)</td> 
    <td>0.?????</td>
    <td>0.?????</td>
    <td>nyx1で実行</td>
  </tr>   
  <tr>
    <td>submission_06111430.csv</td>
    <td>XGB&LR&RF</td> 
    <td>0.88907</td>
    <td>0.?????</td>
    <td>nyx1で実行</td>
  </tr>  
  <tr>
    <td>submission_06131630.csv</td>
    <td>xgb(4,200):mlp:lgb=70:70:3</td> 
    <td>0.8869</td>
    <td>0.?????</td>
    <td>nyx1で実行、_clean_06111230を使用</td>
  </tr>   
  <tr>
    <td>submission_06131630.csv</td>
    <td>xgb(4,200):mlp=1:1</td> 
    <td>0.8892</td>
    <td>0.?????</td>
    <td>nyx1で実行、_clean_06131800を使用。lgbmいらんかった...。</td>
  </tr>     
</table>


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from datetime import datetime

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
#m = xgb.XGBRegressor()
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
#from lightgbm import LGBMClassifier
from sklearn.grid_search import GridSearchCV
from scipy.sparse import csr_matrix
from sklearn.model_selection import cross_val_score

/Users/KojiMinoda/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/KojiMinoda/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
df = pd.read_csv("train_clean_06180000.csv")
df_test = pd.read_csv("test_clean_06180000.csv")

In [77]:
df = pd.read_csv("train_clean_06181111.csv")
df_test = pd.read_csv("test_clean_06181111.csv")

In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219319 entries, 0 to 219318
Columns: 860 entries, driver_gender to location_I395
dtypes: float64(785), int64(75)
memory usage: 1.4 GB


In [66]:
df.columns[df.sum()<30]

Index(['Y', 'location_monroe', 'location_north branford', 'location_avon',
       'location_portland', 'location_hartland', 'location_easton',
       'location_bloomfield', 'location_ridgefield', 'location_suffield',
       'location_wolcott', 'location_simsbury', 'location_granby',
       'location_wilton'],
      dtype='object')

In [ ]:
sc = StandardScaler()
df[["driver_age_raw","population","area","stop_day","X","Y"]]= sc.fit_transform(df[["driver_age_raw","population","area","stop_day","X","Y"]])
df_test[["driver_age_raw","population","area","stop_day","X","Y"]] = sc.fit_transform(df_test[["driver_age_raw","population","area","stop_day","X","Y"]])

In [ ]:
X = df.drop(["is_arrested"],axis=1)
y = df["is_arrested"]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

In [69]:
X = df.drop(["is_arrested",'location_monroe', 'location_north branford', 'location_avon',
       'location_portland', 'location_hartland', 'location_easton',
       'location_bloomfield', 'location_ridgefield', 'location_suffield',
       'location_wolcott', 'location_simsbury', 'location_granby',
       'location_wilton'],axis=1)
y = df["is_arrested"]

In [13]:
y = df["is_arrested"]
X_csr = csr_matrix(df.drop(["is_arrested"],axis=1))
X_train, X_test, y_train, y_test = train_test_split(X_csr, y, train_size=0.8)

/Users/KojiMinoda/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [18]:
param_grid = {"max_depth": [2,3,4],
              "n_estimators":[50,100,300,500],
              #"reg_lambda": [1, 100, 500],
              #"num_leaves": [20, 30, 40],
              "learning_rate": [0.01, 0.1, 0.3]
             }
lgbm_grid = GridSearchCV(estimator=LGBMClassifier(random_state=0),
                 param_grid = param_grid,   
                 scoring="roc_auc", 
                 cv = 10,
                 n_jobs = 8)   
lgbm_grid.fit(X,y) 
lgbm_grid_best = lgbm_grid.best_estimator_
print("Best Model Parameter: ",lgbm_grid.best_params_)

Best Model Parameter:  {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 300}


In [14]:
%%time
lgbc = LGBMClassifier(class_weight="balanced",max_depth=5,n_estimators=500,learning_rate=0.1
                      ,num_leaves=10,min_child_weight=1000)
lgbc.fit(X_train,y_train)
y_predict_lgb = lgbc.predict_proba(X_test)[:,1]
y_train_predict_lgb = lgbc.predict_proba(X_train)[:,1]
print("train data score:",roc_auc_score(y_train.values,y_train_predict_lgb))
print("cross validation score:",roc_auc_score(y_test.values,y_predict_lgb))

NameError: name 'LGBMClassifier' is not defined

In [48]:
%%time
##06180000.csv用パラメータ
xgbc = xgb.XGBClassifier(class_weight="balanced"
                         ,max_depth=7
                         ,min_child_weight=5
                         ,gamma = 0.10
                         ,subsample=0.90
                         ,colsample_bytree=0.8
                         ,scale_pos_weight=1
                         ,reg_alpha=0.0
                         ,learning_rate=0.1)
xgbc.fit(X_train,y_train)
y_predict_xgb = xgbc.predict_proba(X_test)[:,1]
y_train_predict_xgb = xgbc.predict_proba(X_train)[:,1]
print("train data score:",roc_auc_score(y_train.values,y_train_predict_xgb))
print("cross validation score:",roc_auc_score(y_test.values,y_predict_xgb))

train data score: 0.9196806827070264
cross validation score: 0.8871182972647145
CPU times: user 31.6 s, sys: 236 ms, total: 31.9 s
Wall time: 32.3 s


In [48]:
%%time
##06181111.csv用パラメータ
xgbc = xgb.XGBClassifier(class_weight="balanced"
                         ,max_depth=7
                         ,min_child_weight=5
                         ,gamma = 0.10
                         ,subsample=0.90
                         ,colsample_bytree=0.8
                         ,scale_pos_weight=1
                         ,reg_alpha=0.0
                         ,learning_rate=0.1)
xgbc.fit(X_train,y_train)
y_predict_xgb = xgbc.predict_proba(X_test)[:,1]
y_train_predict_xgb = xgbc.predict_proba(X_train)[:,1]
print("train data score:",roc_auc_score(y_train.values,y_train_predict_xgb))
print("cross validation score:",roc_auc_score(y_test.values,y_predict_xgb))

train data score: 0.9196806827070264
cross validation score: 0.8871182972647145
CPU times: user 31.6 s, sys: 236 ms, total: 31.9 s
Wall time: 32.3 s


<table class="table2">
    <caption>縦：min_child_weight,横:max_depth</caption>
  <tr><th></th><th>6</th><th>7</th><th>8</th></tr>
  <tr><td>4</td><td>0.911/0.885</td><td>0.921/0.885</td><td>???/???</td></tr>
  <tr><td>5</td><td>???/0.886</td><td>0./0.887</td><td>0.930/0.886</td></tr>
  <tr><td>6</td><td>???/???</td><td>0.917/0.884</td><td>0.928/0.886</td></tr>
</table>

<table class="table2">
    <caption>縦：min_child_weight,横:max_depth</caption>
  <tr><th></th><th>3</th><th>5</th><th>7</th><th>9</th></tr>
  <tr><td>1</td><td>0.880/0.881</td><td>0.903/0.8857</td><td>???,???</td><td>???,???</td></tr>
  <tr><td>3</td><td>???,???</td><td>0.901/0.886</td><td>0.924/0.885</td><td>0.946/0.883</td></tr>
  <tr><td>5</td><td>0.879/0.880</td><td>0.899/0.885</td><td>0.920/0.887</td><td>???,???</td></tr>
  <tr><td>7</td><td>???? / ????</td><td>0.898/0.884</td><td>???/0.884</td><td>0.932/0.884</td></tr>
</table>

<table class="table2">
    <caption>縦：subsample,横:col_sample_tree</caption>
  <tr><th></th><th>0.6</th><th>0.7</th><th>0.8</th><th>0.9</th></tr>
  <tr><td>0.6</td><td>???,???</td><td>0.904/0.883</td><td>0.930/0.884</td><td>0.957/0.885</td></tr>
  <tr><td>0.7</td><td>0.880/0.874</td><td>???/??? </td><td>0.922/0.883</td><td>0.945/0.885</td></tr>
  <tr><td>0.8</td><td>???,???</td><td>0.901/0.881</td><td>0.919/0.884</td><td>0.939/0.884</td></tr>
  <tr><td>0.9</td><td>???? / ????</td><td></td><td>0.916/0.883</td><td>0.933/0.884</td></tr>
</table>

<table class="table2">
    <caption>縦：min_child_weight,横:max_depth</caption>
  <tr><th></th><th>8</th><th>9</th><th>10</th></tr>
  <tr><td>2</td><td>0.936/0.884</td><td>0.950/0.883</td><td>0.961/0.882</td></tr>
  <tr><td>3</td><td>0.933/0.884 </td><td>0.944/0.886</td><td>0.954/0.883</td></tr>
  <tr><td>4</td><td>0.931/0.883</td><td>0.945/0.882</td><td>0.951/0.883</td></tr>
</table>

<table class="table2">
    <caption>縦：min_child_weight,横:max_depth</caption>
  <tr><th></th><th>3</th><th>5</th><th>7</th><th>9</th></tr>
  <tr><td>1</td><td>???,???</td><td>0.904/0.883</td><td>0.930/0.884</td><td>0.957/0.885</td></tr>
  <tr><td>3</td><td>0.880/0.874</td><td>???/??? </td><td>0.922/0.883</td><td>0.945/0.885</td></tr>
  <tr><td>5</td><td>???,???</td><td>0.901/0.881</td><td>0.919/0.884</td><td>0.939/0.884</td></tr>
  <tr><td>7</td><td>???? / ????</td><td></td><td>0.916/0.883</td><td>0.933/0.884</td></tr>
</table>

<table class="table2">
    <caption>06121820</caption>
  <tr><th></th><th>3</th><th>4</th><th>5</th><th>6</th></tr>
  <tr><td>100</td><td>???,???</td><td>??? / ???</td><td>0.8990 / 0.8768</td><td>???,???</td></tr>
  <tr><td>200</td><td>???,???</td><td>??? / </td><td>???,???</td><td>???,???</td></tr>
  <tr><td>300</td><td>???,???</td><td>??? / >???</td><td>???,???</td><td>???,???</td></tr>
    <tr><td>500</td><td>???? / ????</td><td>0.9181 / 0.8734</td><td>?? / ???</td><td>???,???</td></tr>
  <tr><td>700</td><td>0.9054 / 0.8726</td><td>???? / ????</td><td>??? / ???</td><td>???,???</td>
</table>


<table class="table2">
    <caption>06111230</caption>
  <tr><th></th><th>3</th><th>4</th><th>5</th><th>6</th></tr>
  <tr><td>100</td><td>???,???</td><td>0.8903 / 0.8768</td><td>??? / 0.8872?</td><td>???,???</td></tr>
  <tr><td>200</td><td>???,???</td><td>??? / </td><td>???,???</td><td>???,???</td></tr>
  <tr><td>300</td><td>???,???</td><td>0.9093 / 0.8793</td><td>???,???</td><td>???,???</td></tr>
    <tr><td>500</td><td>0.9004,0.8791</td><td>0.9184 / 0.8811</td><td>0.9376 / 0.8699</td><td>???,???</td></tr>
  <tr><td>700</td><td>???,???</td><td>0.9272 / 0.8809</td><td>??? / ???</td><td>???,???</td>
</table>

In [68]:
%%time
mlp = MLPClassifier(alpha=1e-2,hidden_layer_sizes=8, random_state=1)
mlp.fit(X_train,y_train)
y_predict_mlp = mlp.predict_proba(X_test)[:,1]
y_train_predict_mlp = mlp.predict_proba(X_train)[:,1]
print("train data score:",roc_auc_score(y_train.values,y_train_predict_mlp))
print("cross validation score:",roc_auc_score(y_test.values,y_predict_mlp))

train data score: 0.8915722912690084
cross validation score: 0.8773911586511993
CPU times: user 24.7 s, sys: 405 ms, total: 25.1 s
Wall time: 12.7 s


In [73]:
%%time
lr = LogisticRegression(class_weight="balanced",C=0.1)
lr.fit(X_train,y_train)
y_predict_lr = lr.predict_proba(X_test)[:,1]
y_train_predict_lr = lr.predict_proba(X_train)[:,1]
print("train data score:",roc_auc_score(y_train.values,y_train_predict_lr))
print("cross validation score:",roc_auc_score(y_test.values,y_predict_lr))

train data score: 0.8804598466356508
cross validation score: 0.8753269408913045
CPU times: user 2.44 s, sys: 59.7 ms, total: 2.5 s
Wall time: 2.44 s


In [33]:
%%time
forest = RandomForestClassifier(max_depth=3,n_estimators=200,class_weight="balanced")
forest.fit(X_train,y_train)
y_predict_forest = forest.predict_proba(X_test)[:,1]
y_train_predict_forest = forest.predict_proba(X_train)[:,1]
print("train data score:",roc_auc_score(y_train.values,y_train_predict_forest))
print("cross validation score:",roc_auc_score(y_test.values,y_predict_forest))

train data score: 0.8619618898679375
cross validation score: 0.8436802550260822
CPU times: user 26.7 s, sys: 660 ms, total: 27.4 s
Wall time: 12.2 s


In [74]:
%%time
adbc = AdaBoostClassifier(n_estimators=100)
adbc.fit(X_train,y_train)
y_predict_adb = adbc.predict_proba(X_test)[:,1]
y_train_predict_adb = adbc.predict_proba(X_train)[:,1]
print("train data score:",roc_auc_score(y_train.values,y_train_predict_adb))
print("cross validation score:",roc_auc_score(y_test.values,y_predict_adb))

train data score: 0.8782988742810675
cross validation score: 0.8705607415770089
CPU times: user 56.8 s, sys: 3.1 s, total: 59.9 s
Wall time: 31 s


<table>
  <caption>clean_06121840</caption>
  <thead>
  <tr>
    <td>あるふぁ</td>
    <td>隠れ層サイズ</td>
    <td>train data score</td>  
    <td>cross validation score</td>       
  </tr>
  </thead>
  <tr>
    <td>1e-2</td>
    <td>10</td> 
    <td>0.8877</td>
    <td>0.8893</td>
  </tr>
  <tr>
    <td>1e-2</td>
    <td>6</td> 
    <td>0.8887</td>
    <td>0.8825</td>
  </tr> 
</table>
<table>
  <caption>clean_06111230</caption>
  <thead>
  <tr>
    <td>あるふぁ</td>
    <td>隠れ層サイズ</td>
    <td>train data score</td>  
    <td>cross validation score</td>       
  </tr>
  </thead>
  <tr>
    <td>1e-5</td>
    <td>50</td> 
    <td>0.96</td>
    <td>0.82</td>
  </tr> 
  <tr>
    <td>1e-5</td>
    <td>30</td> 
    <td>0.946</td>
    <td>0.8284</td>
  </tr>    
  <tr>
    <td>1e-5</td>
    <td>20</td> 
    <td>0.9258</td>
    <td>0.8543</td>
  </tr>   
  <tr>
    <td>1e-5</td>
    <td>10</td> 
    <td>0.9072</td>
    <td>0.8718</td>
  </tr> 
  <tr>
    <td>1e-5</td>
    <td>6</td> 
    <td>0.8934</td>
    <td>0.8802</td>
  </tr>   
  <tr>
    <td>1e-3</td>
    <td>6</td> 
    <td>0.8923</td>
    <td>0.8828</td>
  </tr>
  <tr>
    <td>1e-2</td>
    <td>6</td> 
    <td>0.8910</td>
    <td>0.8834</td>
  </tr>   
  <tr>
    <td>1e-5</td>
    <td>3</td> 
    <td>0.8747</td>
    <td>0.8612</td>
  </tr>   
</table>

In [76]:
#lr,xgb,mlp,lgbm
rate = np.array([15,2])
roc_auc_score(y_test.values,np.dot(rate,np.array([y_predict_xgb
                                                  ,y_predict_mlp]))/np.sum(rate))

0.887306632522485

In [10]:
X_final_test =  csr_matrix(df_test)

In [224]:
%%time
lgbc_final = LGBMClassifier(class_weight="balanced",n_estimators=30,learning_rate=0.3
                      ,reg_lambda=40,num_leaves=45)
lgbc_final.fit(X,y)

CPU times: user 2.25 s, sys: 568 ms, total: 2.82 s
Wall time: 2.82 s


In [7]:
%%time
xgb_final = xgb.XGBClassifier(class_weight="balanced"
                         ,max_depth=9
                         ,min_child_weight=3
                         ,gamma = 0.1
                         ,subsample=0.90
                         ,colsample_bytree=0.8
                         ,scale_pos_weight=1
                         ,reg_alpha=0.0
                         ,learning_rate=0.1)
xgb_final.fit(X_csr,y)

CPU times: user 39 s, sys: 149 ms, total: 39.1 s
Wall time: 39.2 s


In [8]:
%%time
mlp_final = MLPClassifier(alpha=1e-2,hidden_layer_sizes=6, random_state=1)
mlp_final.fit(X_csr,y)

CPU times: user 31.8 s, sys: 512 ms, total: 32.3 s
Wall time: 16.3 s


In [11]:
y_final_predict_xgb = xgb_final.predict_proba(X_final_test)[:,1]
y_final_predict_mlp = mlp_final.predict_proba(X_final_test)[:,1]
rate = np.array([2,1])
y_final_predict = np.dot(rate,np.array([y_final_predict_xgb,y_final_predict_mlp]))/np.sum(rate)

In [12]:
y_final_predict = np.round(y_final_predict,6)

In [13]:
np.average(y_final_predict)

0.022711434878822052

In [14]:
submission = pd.DataFrame(data=y_final_predict, columns=['is_arrested'])
submission.head(3)

,is_arrested
0,0.004605
1,0.015865
2,0.320153


In [15]:
submission.to_csv("submission.csv",index=False)